In [5]:
"""將tensorflow檔轉成tensorflow lite，使機器負載量變小"""
from tensorflow import lite
from tensorflow.keras import models

# Parameters
keras_model_filename = './h5/appendEnergyTT_wen.h5' #訓練好的模型
tflite_filename = './tflite/appendEnergyTT_wen.tflite' #預建置檔案

# Convert model to TF Lite model
model = models.load_model(keras_model_filename) #載入本來的模型
converter = lite.TFLiteConverter.from_keras_model(model) #將模型載入轉換器
tflite_model = converter.convert() #進行轉換
open(tflite_filename, 'wb').write(tflite_model) #輸出轉換後的模型

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp4qdwm0uf\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp4qdwm0uf\assets


235300

In [1]:
#slide_window=0.25
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa

# This gets called every 0.5 seconds
def sound(window,s,m):
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) > 3000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=8000, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=True,#True的話，第0個倒頻譜係數被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if list_val_max >=0 :
           #data_ho.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
            data.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
             
         
#main
# Parameters
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/appendEnergyTT_wen.tflite'
words = ['贅詞', '雜音']#答案對應到的字詞
data = []
data_ho = []
start = 0 #一開始的索引值
end = 2000 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 180 #讀音檔的總時間
sample_rate = 8000 #取樣率

#載入音檔
y, sr = librosa.load("./desktop/chen.wav",sr=sample_rate,duration=duration) 

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


while True:
    s = s + 0.25 #增加秒數
    if(s == 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    #window[:4000] = window[4000:] #把音訊載入window
    #window[4000:] = y[start:end] #把音訊載入window
    window[0:2000] = window[2000:4000]
    window[2000:4000] = window[4000:6000]
    window[4000:6000] = window[6000:8000]
    window[6000:8000] = y[start:end]
    sound(window,s,m) #呼叫sound()

    if(end == (8000 * duration)): #如果移動到最後，break
        break
    
    start = start + 2000 #向後移動
    end = end + 2000 #向後移動    
    
    
for i in range(len(data)):
        print(data[i])
        
print(len(data))

0分1.75秒，出現:贅詞，預測值:0.8790212869644165
0分2.0秒，出現:贅詞，預測值:0.9999096393585205
0分2.25秒，出現:贅詞，預測值:0.9999978542327881
0分2.5秒，出現:贅詞，預測值:0.9999996423721313
0分2.75秒，出現:贅詞，預測值:0.9999978542327881
0分3.0秒，出現:贅詞，預測值:0.997046172618866
0分3.25秒，出現:贅詞，預測值:0.9742343425750732
0分3.5秒，出現:贅詞，預測值:1.0
0分3.75秒，出現:贅詞，預測值:1.0
0分4.0秒，出現:贅詞，預測值:1.0
0分4.25秒，出現:贅詞，預測值:0.9999998807907104
0分4.5秒，出現:贅詞，預測值:0.9999991655349731
0分4.75秒，出現:贅詞，預測值:0.9999198913574219
0分5.0秒，出現:贅詞，預測值:0.9999909400939941
0分5.25秒，出現:贅詞，預測值:0.9999980926513672
0分5.5秒，出現:贅詞，預測值:1.0
0分5.75秒，出現:贅詞，預測值:0.9999990463256836
0分6.0秒，出現:贅詞，預測值:0.9999649524688721
0分6.25秒，出現:贅詞，預測值:0.9988560676574707
0分6.5秒，出現:贅詞，預測值:0.9999983310699463
0分6.75秒，出現:贅詞，預測值:0.9999998807907104
0分7.0秒，出現:贅詞，預測值:0.9999998807907104
0分7.25秒，出現:贅詞，預測值:0.9995267391204834
0分7.5秒，出現:贅詞，預測值:0.9954250454902649
0分7.75秒，出現:贅詞，預測值:0.9999998807907104
0分8.0秒，出現:贅詞，預測值:1.0
0分8.25秒，出現:贅詞，預測值:0.9999668598175049
0分8.5秒，出現:贅詞，預測值:0.9999856948852539
0分8.75秒，出現:贅詞，預測值:0.9999935626983643
0分9.0秒，出現:贅詞，預測值: